In [1]:
import sys
import json
import cobra
import cplex
import cobrakbase
import escher
from escher import Builder


cobrakbase 0.2.7


In [2]:
#Put the path to ModelSEEDpy on your machine here
sys.path.append("/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh")
#import modelseedpy.fbapkg
from modelseedpy import MSPackageManager


In [3]:

# file = open('/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh/examples/plant_map.json',)
# plantmap = file.read()
# builder = Builder(map_json=plantmap)
# builder.reaction_scale=[
#         { "type": 'value', "value": 0, "color": '#dcdcdc', "size": 10 },
#         { "type": 'value', "value": 0.000001, "color": '#9696ff', "size": 25 },
#         { "type": 'value', "value": 1, "color": '#ff0000', "size": 35 },
#         { "type": 'value', "value": 20, "color": '#209123', "size": 50 },
#     ]
# builder

In [4]:
kbase_api = cobrakbase.KBaseAPI()
#model = kbase_api.get_from_ws("C4_Calvin_Zmays_Model",25427)
model =cobra.io.read_sbml_model('C4_HS_Zmays_Model_v2.xml')
#model =cobra.io.read_sbml_model('C4_Calvin_Zmays_Model_v8.xml')
model.solver = 'optlang-cplex'

In [5]:

#Setting model to complete media
#media = kbase_api.get_from_ws("Carbon-D-Glucose","KBaseMedia")
pkgmgr = MSPackageManager.get_pkg_mgr(model,1)
pkgmgr.getpkg("KBaseMediaPkg",1).build_package(None)

#Reading reaction protein abundances
data = ""
with open('/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh/examples/ReactionProtein.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_proteins = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        for i in range(2, len(array)):
            if headers[i] not in reaction_proteins:
                reaction_proteins[headers[i]] = dict()
            reaction_proteins[headers[i]][array[0]] = abs(float(array[i]))

#Reading reaction kcat values
data = ""
with open('/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh/examples/KCats.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_kcat = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        reaction_kcat[array[0]] = abs(float(array[1]))

#Reading measured reaction fluxes
data = ""
with open('measured_Claudia4.txt', 'r') as file:
#with open('/Users/ffoflonker/Documents/Command_line/ModelSEEDpy-csh/examples/MeasuredReaction.txt', 'r') as file:
    data = file.read()
lines = data.split("\n")
reaction_measures = {}
headers = None
for line in lines:
    if headers == None:
        headers = line.split("\t")
    else:
        array = line.split("\t")
        if array[1] not in reaction_measures:
                reaction_measures[array[1]] = dict()
        rxnid = array[0]
        if rxnid[-1:] != "z":
            rxnid += "0"
        else:
            rxnid =rxnid.rstrip(rxnid[-1])
            rxnid += "d1"
        reaction_measures[array[1]][rxnid] = abs(float(array[2]))

print(reaction_measures)

{'Mature_leaf': {'rxn00102_d1': 0.031916667, 'rxn20508_d1': 0.031916667, 'rxn00249_d1': 0.031916667, 'rxn00161_d0': 0.031916667, 'rxn00151_d1': 0.031916667, 'rxn00018_d0': 0.031916667, 'rxn01100_d0': 0.031916667, 'rxn00782_d0': 0.031916667, 'rxn00781_d0': 0.002125, 'rxn01345_d0': 0.031916667, 'rxn01200_d0': 0.031916667, 'rxn01111_d0': 0.031916667, 'rxn02251_d0': 0.0005, 'rxn00980_d0': 0.0005, 'rxn00333_x0': 0.0005, 'rxn00275_x0': 0.0005, 'rxn00692_m0': 0.0005, 'rxn00424_x0': 0.0005, 'rxn01011_x0': 0.0005, 'rxn00908_m0': 0.0005, 'rxn15364_c0': 0.002125, 'rxn01334_c0': 0.002125, 'rxn01100_c0': 0.002125, 'rxn00781_c0': 0.002125, 'rxn01106_c0': 0.002125, 'rxn00459_c0': 0.002125, 'rxn00148_c0': 0.002125, 'rxn00747_c0': 0.002125, 'rxn15298_c0': 0.002125, 'rxn15493_c0': 0.002125, 'rxn15494_c0': 0.002125, 'rxn20583_d0': 0.002125, 'rxn15364_d0': 0.002125, 'rxn01975_d0': 0.0004875, 'rxn01476_d0': 0.0004875, 'rxn01115_d0': 0.0004875, 'rxn11040_d0': 0.0004875, 'rxn29919_d0': 0.0004875, 'rxn01975_c

In [6]:
#Applying flexible biomass constraints
protein = 0.0948
protein_flux = -1*(0.2-protein)
pkgmgr.getpkg("FlexibleBiomassPkg",1).build_package({"bio_rxn_id":"bio1_biomass","use_rna_class":[-0.75,0.75],
    "use_dna_class":[-0.75,0.75],
    "use_protein_class":[-0.1,0.1],
    "use_energy_class":[-0.1,0.1]})

with model:
    #Applying proteome fitting constraints
    pkgmgr.getpkg("ProteomeFittingPkg",1).build_package({
        "rxn_proteome":reaction_proteins["Mature_leaf"],
        "flux_values":reaction_measures["Mature_leaf"],
        "kcat_values":reaction_kcat,
        "prot_coef" : protein,#Set to the fraction of the cell that is protein
        "totalflux" : 10,#Set to one if we're fitting flux magnitude rather than actual flux to flux measurements
        "kcat_coef" : 0.02,#kapp = kcat_coef * kcat
        "obj_kfit":1,#Coefficient on kcat fitting terms in objective function
        "obj_kvfit":1,#Coefficient on kinetic expression fitting terms in objective function
        "obj_vfit":100,#Coefficient on flux measurement terms in objective function
        "set_objective":1
    })
    
    #Constraining biomass flux to reference value
    rxn = model.reactions.get_by_id("bio1_biomass")
    rxn.upper_bound = 0.006395833
    rxn.lower_bound = 0.006395833
    
    #Printing LP file
    with open('SingleTissueProteomeData.lp', 'w') as out:
        out.write(str(model.solver))
            #Optimizing
    sol=model.optimize()
    #model.summary()
#     builder.model = model
#     builder.reaction_data = sol.fluxes
    #Print solution:
    #Key variables:
    #rxn#####_c0_kapp - k apparent for reaction 29
    #rxn#####_c0_kvfit - fit of reaction flux to equation kapp*[protein]
    #rxn#####_c0_kfit - fit of kapp to 0.02*kcat
    #rxn#####_c0_tf - total flux through reaction
    #rxn#####_c0_vfit - fit to measured value for this reaction
    #Objective value is to minimize the sum of squares for kvfit, kfit, and vfit variables
    


In [7]:
sol.fluxes['bio1_biomass']

0.006395833

In [8]:
sol.fluxes.to_csv('single_prot_sol.csv')

In [9]:
model.reactions.rxn01116_d0

Reaction identifier,rxn01116_d0
Name,D-Ribulose-5-phosphate 3-epimerase
Memory address,0x07fa3afef9990
Stoichiometry,cpd00171_d0 <=> cpd00198_d0 D-Ribulose5-phosphate <=> D-Xylulose5-phosphate
GPR,GRMZM2G026807 and GRMZM2G178960 and GRMZM2G083102
Lower bound,-1000.0
Upper bound,1000.0
